# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,waitangi,-43.9535,-176.5597,12.12,84,100,3.13,NZ,1700338616
1,1,remire-montjoly,4.9167,-52.2667,29.02,79,75,5.14,GF,1700338618
2,2,cidreira,-30.1811,-50.2056,21.07,83,99,5.96,BR,1700338619
3,3,praia da vitoria,38.7333,-27.0667,19.31,82,40,7.32,PT,1700338620
4,4,palmer,42.1584,-72.3287,8.90,53,20,8.75,US,1700338369


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600, 
    size = "Humidity",
    color = "City"
)


# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [7]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_df = city_data_df.loc[city_data_df["Humidity"] <= 60]


# Drop any rows with null values
filtered_df = filtered_df.dropna()


# Display sample data
filtered_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,palmer,42.1584,-72.3287,8.90,53,20,8.75,US,1700338369
9,9,tabas,33.5959,56.9244,17.12,39,0,1.03,IR,1700338628
16,16,haiku-pauwela,20.9219,-156.3051,26.76,53,20,2.06,US,1700338638
18,18,bethel,41.3712,-73.4140,11.20,37,0,4.12,US,1700338640
23,23,albany,42.6001,-73.9662,6.36,53,0,2.24,US,1700338511


### Step 3: Create a new DataFrame called `hotel_df`.

In [8]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""


# Display sample data
hotel_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,waitangi,-43.9535,-176.5597,12.12,84,100,3.13,NZ,1700338616,
1,1,remire-montjoly,4.9167,-52.2667,29.02,79,75,5.14,GF,1700338618,
2,2,cidreira,-30.1811,-50.2056,21.07,83,99,5.96,BR,1700338619,
3,3,praia da vitoria,38.7333,-27.0667,19.31,82,40,7.32,PT,1700338620,
4,4,palmer,42.1584,-72.3287,8.90,53,20,8.75,US,1700338369,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [10]:
# Set parameters to search for a hotel
radius = 10000
params  = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    Lat = row["Lat"]
    Lng = row["Lng"]
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{Lng},{Lat},{radius}"
    params["bias"] = f"proximity:{Lng},{Lat}"
    
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
# Display sample data
hotel_df

Starting hotel search
waitangi - nearest hotel: Hotel Chathams
remire-montjoly - nearest hotel: Complexe Belova
cidreira - nearest hotel: Hotel Castelo
praia da vitoria - nearest hotel: Salles
palmer - nearest hotel: No hotel found
iqaluit - nearest hotel: Frobisher Inn
bredasdorp - nearest hotel: Victoria Hotel
tezu - nearest hotel: No hotel found
georgetown - nearest hotel: Page 63 hostel
tabas - nearest hotel: هتل امیر
port mathurin - nearest hotel: Escale Vacances
mejit - nearest hotel: No hotel found
edinburgh of the seven seas - nearest hotel: No hotel found
college - nearest hotel: BEST WESTERN PLUS Pioneer Park Inn
arteche - nearest hotel: No hotel found
grytviken - nearest hotel: No hotel found
haiku-pauwela - nearest hotel: Inn At Mama's Fish House
karratha - nearest hotel: Karratha International Hotel
bethel - nearest hotel: Hampton Inn Danbury
newman - nearest hotel: No hotel found
ola - nearest hotel: No hotel found
margaret river - nearest hotel: Margaret River Hotel
aasi

yeppoon - nearest hotel: Blossom's on Seaspray
touros - nearest hotel: Pousada Atlântico
gadzhiyevo - nearest hotel: Оазис
bubaque - nearest hotel: Chez Julio
dudley park - nearest hotel: Comfort inn
glace bay - nearest hotel: No hotel found
illizi - nearest hotel: بوناقة
nema - nearest hotel: No hotel found
castillos - nearest hotel: A mi gente
paradip garh - nearest hotel: Hotel Sea Pearl
nuuk - nearest hotel: Nordbo
coahuayana de hidalgo - nearest hotel: No hotel found
anan - nearest hotel: ロイヤルガーデンホテル
east london - nearest hotel: No hotel found
pundong - nearest hotel: Puri Ganjuran
luderitz - nearest hotel: Seaview Hotel Zum Sperrgebiet
baishishan - nearest hotel: No hotel found
astrakhan - nearest hotel: Omega
mutoko - nearest hotel: No hotel found
soultz-haut-rhin - nearest hotel: Château d'Anthès
champerico - nearest hotel: Hotel y Restaurante El Submarino
faulquemont - nearest hotel: Le Châtelain
bathsheba - nearest hotel: Atlantis Hotel
solleftea - nearest hotel: Hotell Appel

broome - nearest hotel: No hotel found
loikaw - nearest hotel: Nawaday Hotel
nogliki - nearest hotel: Ноглики
jatani - nearest hotel: HOTEL SURYA MUKHI
businga - nearest hotel: Hôtel Roho
ts'khinvali - nearest hotel: No hotel found
puerto baquerizo moreno - nearest hotel: Dory's House
tha khanon - nearest hotel: No hotel found
borogontsy - nearest hotel: No hotel found
laguna - nearest hotel: Holiday Inn Express & Suites
luena - nearest hotel: No hotel found
athens - nearest hotel: Eridanus
beinamar - nearest hotel: No hotel found
vila franca do campo - nearest hotel: Vinha d'Areia Beach Hotel
kiamba - nearest hotel: Alquiza's Bahay Bakasyunan Tourist Inn
greymouth - nearest hotel: Recreation Hotel
vanimo - nearest hotel: CBC Guesthouse
oistins - nearest hotel: The Abidah By Accra
krasnogorskiy - nearest hotel: No hotel found
muhos - nearest hotel: No hotel found
la mira - nearest hotel: Hotel La Loma
gorontalo - nearest hotel: Grand City Hotel
ayios konstandinos - nearest hotel: Marpe

noosa heads - nearest hotel: Costa Nova Apartments
hauterive - nearest hotel: Auberge Bourbonnaise
wewak - nearest hotel: Village Inn
posto fiscal rolim de moura - nearest hotel: No hotel found
galliano - nearest hotel: Fairfield Inn & Suites by Marriott Cut Off-Galliano
jijiga - nearest hotel: Africa Hotel
pemangkat - nearest hotel: Hotel Kelapa Gading


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,waitangi,-43.9535,-176.5597,12.12,84,100,3.13,NZ,1700338616,Hotel Chathams
1,1,remire-montjoly,4.9167,-52.2667,29.02,79,75,5.14,GF,1700338618,Complexe Belova
2,2,cidreira,-30.1811,-50.2056,21.07,83,99,5.96,BR,1700338619,Hotel Castelo
3,3,praia da vitoria,38.7333,-27.0667,19.31,82,40,7.32,PT,1700338620,Salles
4,4,palmer,42.1584,-72.3287,8.90,53,20,8.75,US,1700338369,No hotel found
...,...,...,...,...,...,...,...,...,...,...,...
579,579,wewak,-3.5534,143.6268,25.66,89,23,3.23,PG,1700339406,Village Inn
580,580,posto fiscal rolim de moura,-13.0833,-62.2667,39.96,22,73,2.41,BR,1700339409,No hotel found
581,581,galliano,29.4422,-90.2992,24.00,57,0,5.14,US,1700339410,Fairfield Inn & Suites by Marriott Cut Off-Gal...
582,582,jijiga,9.3500,42.8000,18.68,85,76,2.85,ET,1700339411,Africa Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [12]:
%%capture --no-display

# Configure the map plot
map_plot_2 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600, 
    size = "Humidity",
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)
# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)